<h1 style="color:blue;text-align:center">Docker Essentials: A Developer Introduction</h1>

# Objectives
 + What are containers and their benefits.
 + How to find and run Docker containers from the Dorcker Hub.
 + How to create your first Docker container and understand the layering and union file system.
 + The best practice when developing Dockerfiles.
 + How to solve problems of orchestration, scaling, high availability, and service discovery.
 

# What are Containers?
 + A group of processes run in isolation.
   + All processes must be able to run on the shared kernel.
 + Each container has its own set of 'namespaces'(isolated view).
   + **PID**- process ID
   + **USER**- user and group IDS.
   + **UTS**- hostname and domain name.
   + **NS**- mount points.
   + **NET**- Network devices, stacks, ports
   + **IPC**- inter-process-communications, message queues
 + **cgroups**- controls limits and monitoring of resources 

<div>
    <img src="s5.png" width=300 height=300>
    <img src="s6.png" width=300 height=300>
    <img src="s7.png" width=300 height=300>
</div>

<h1 style="color:blue;text-align:center">VM vs Container</h1>

<img src="s1.png" width=600 height=600>

+ container run on top of the base kernel and they use linux namespace they are fast as they do not use a full OS.
+ we get the benefit of VM(isolation) without the cost of heaviness.
+ container can be run on top of VM.

# What is Docker?
+ At its core, Docker is tooling to manage containers.
  + simplified existing technology to enable it for the masses.
+ Enable developers to use containers for their applications.
  +Package dependencies with container: "build once, run anywhere".

# Advantages of Docker
+ No more "works on my machine".
+ Lightweight and fast.
+ Better resource utilization.
  + can fit far more containers than VMs into a host.
+ Standard developer to operations interface.
+ Ecosystem and tooling.

<p style="color:green;border:2px solid black;padding:10px">Containers are just a process (or a group of processes) running in isolation, which is achieved with Linux namespaces and control groups. Linux namespaces and control groups are features that are built into the Linux kernel. Other than the Linux kernel itself, there is nothing special about containers.
What makes containers useful is the tooling that surrounds them. The labs in this course use Docker, which has been the understood standard tool for using containers to build applications. <span style="color:blue">Docker provides developers and operators with a friendly interface to build, ship, and run containers on any environment.</span>
In the first part of this lab, run your first container, and learn how to inspect it. You will be able to witness the namespace isolation that you acquire from the Linux kernel.</p>

# Docker Images
+ Tar file containing a container's filesystem and metadata
+ reasons that we create images is, we can share and redistribute our images to other users or other environments

# Docker Registry
+ we can push our images or pull images from registry
+ default registry: **Docker Hub**
    + public and free for public images.
    + many pre-packaged images are available
+ **private registry**
    + self-host or cloud provider options
    + docker registry is also available as a docker image
    + <code style="color:red"> docker run registry</code> command is used to run a private registry

# Creating a Docker image- with Docker build
  + create a **Dockerfile**
     + List of instructions for how to construct the container.
  + <code style="color:red">docker build -f Dockerfile</code> with this command docker engine will build a docker image

<img src="s2.png" width=600 height=600>

+ each line correspond to a layer.
+ every layer is build on to of the layers before. if we make a change in line 2, line 2 can resue the line 3 & 4(from casch) and only rebuild the layer 2.

<img src="s4.png" width=600 height=700>

# Creating a custom images

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello():
    return "hello world!"

if __name__ == "__main__":
    app.run(host="0.0.0.0")


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.0.103:5000/ (Press CTRL+C to quit)


# contains of the Dockerfile for creating image

<code style="color:red">FROM python:3.6.1-alpine
RUN pip install --upgrade pip
RUN pip install flask
CMD ["python","app.py"]
COPY app.py /app.py</code>

Let's understand the commands in the Dockerfile line by line.

       
+ FROM python:3.6.1-alpine
   + This is the starting point for your Dockerfile. Every Dockerfile typically starts with a FROM line that is the starting image to build your layers on top of. In this case, you are selecting the python:3.6.1-alpine base layer because it already has the version of Python and pip that you need to run your application. The alpine version means that it uses the alpine distribution, which is significantly smaller than an alternative flavor of Linux. A smaller image means it will download (deploy) much faster, and it is also more secure because it has a smaller attack surface.

        Here you are using the 3.6.1-alpine tag for the Python image. Look at the available tags for the official Python image on the Docker Hub. It is best practice to use a specific tag when inheriting a parent image so that changes to the parent dependency are controlled. If no tag is specified, the latest tag takes effect, which acts as a dynamic pointer that points to the latest version of an image.

        For security reasons, you must understand the layers that you build your docker image on top of. For that reason, it is highly recommended to only use official images found in the Docker Hub, or noncommunity images found in the Docker Store. These images are vetted to meet certain security requirements, and also have very good documentation for users to follow. You can find more information about this Python base image and other images that you can use on the Docker store.

        For a more complex application, you might need to use a FROM image that is higher up the chain. For example, the parent Dockerfile for your Python application starts with FROM alpine, then specifies a series of CMD and RUN commands for the image. If you needed more control, you could start with FROM alpine (or a different distribution) and run those steps yourself. However, to start, it's recommended that you use an official image that closely matches your needs.
+ RUN pip install flask
  + The RUN command executes commands needed to set up your image for your application, such as installing packages, editing files, or changing file permissions. In this case, you are installing Flask. The RUN commands are executed at build time and are added to the layers of your image.
        CMD ["python","app.py"]

      CMD is the command that is executed when you start a container. Here, you are using CMD to run your Python applcation.

       There can be only one CMD per Dockerfile. If you specify more than one CMD, then the last CMD will take effect. The parent python:3.6.1-alpine also specifies a CMD (CMD python2). You can look at the Dockerfile for the official python:alpine image.

       You can use the official Python image directly to run Python scripts without installing Python on your host. However, in this case, you are creating a custom image to include your source so that you can build an image with your application and ship it to other environments.
+ COPY app.py /app.py

    + This line copies the app.py file in the local directory (where you will run docker image build) into a new layer of the image. This instruction is the last line in the Dockerfile. Layers that change frequently, such as copying source code into the image, should be placed near the bottom of the file to take full advantage of the Docker layer cache. This allows you to avoid rebuilding layers that could otherwise be cached. For instance, if there was a change in the FROM instruction, it will invalidate the cache for all subsequent layers of this image. You'll see this little later in this lab.

        It seems counter-intuitive to put this line after the CMD ["python","app.py"] line. Remember, the CMD line is executed only when the container is started, so you won't get a file not found error here.

        And there you have it: a very simple Dockerfile. See the full list of commands that you can put into a Dockerfile. Now that you've defined the Dockerfile, you'll use it to build your custom docker image.

 


In [ ]:
! sudo docker image build -t python-hello-world . # to create an image
!  docker run -p 5001:5000 -d python-hello-world # run the docker image
! sudo docker image ls # show all images
! sudo docker container ls # show all container
! docker container logs [container id] 
! docker stop [container id] # stop a running container
! docker  rm [container id]  # remove container
! docker system prune. # removes containers that are already stopped

The -p flag maps a port running inside the container to your host. In this case, you're mapping the Python app running on port 5000 inside the container to port 5001 on your host. Note that if port 5001 is already being used by another application on your host, you might need to replace 5001 with another value, such as 5002.

# push to a central registry



    Navigate to Docker Hub and create a free account if you haven't already.

    For this lab, you will use the Docker Hub as your central registry. Docker Hub is a free service to publicly store available images. You can also pay to store private images.

    Most organizations that use Docker extensively will set up their own registry internally. To simplify things, you will use  Docker Hub, but the following concepts apply to any registry.

    Log in to the Docker registry account by entering docker login on your terminal:

    $ docker login
    Login with your Docker ID to push and pull images from Docker Hub. If you don't have a Docker ID, head over to https://hub.docker.com to create one.
    Username: 

    Tag the image with your username.

    The Docker Hub naming convention is to tag your image with [dockerhub username]/[image name]. To do this, tag your previously created image python-hello-world to fit that format.

    $ docker tag python-hello-world [dockerhub username]/python-hello-world

    After you properly tag the image, use the docker push command to push your image to the Docker Hub registry:

    $ docker push jzaccone/python-hello-world
    The push refers to a repository [docker.io/jzaccone/python-hello-world]
    2bce026769ac: Pushed 
    64d445ecbe93: Pushed 
    18b27eac38a1: Mounted from library/python 
    3f6f25cd8b1e: Mounted from library/python 
    b7af9d602a0f: Mounted from library/python 
    ed06208397d5: Mounted from library/python 
    5accac14015f: Mounted from library/python 
    latest: digest: sha256:508238f264616bf7bf962019d1a3826f8487ed6a48b80bf41fd3996c7175fd0f size: 1786

    Check your image on Docker Hub in your browser.

    Navigate to Docker Hub and go to your profile to see your uploaded image.

    Now that your image is on Docker Hub, other developers and operators can use the docker pull command to deploy your image to other environments.

    Remember: Docker images contain all the dependencies that they need to run an application within the image. This is useful because you no longer need to worry about environment drift (version differences) when you rely on dependencies that are installed on every environment you deploy to. You also don't need to follow more steps to provision these environments. Just one step: install docker, and that's it.


<h1 style="color:blue;text-align:center">Some useful <span style="color:green">Docker</span> commands</h1>

In [ ]:
sudo docker run docker/whalesay cowsay Hello-World 
 # print docker logo on terminal
sudo docker run nginx
## start container of the image nginx. if the image not present it will 
 ## pull the image from the docker hub only once.
docker ps -a 
# show all the running container(running or stopped)
docker ps
# only running container
docker stop [container name or container id] # to stop a running container
docker rm [container name or container id] # remove previously stopped 
# container 
docker images # show all the images
docker rmi [container name or container id] # delete image. but the 
# container of that image should be stopped before it could be deleted.
docker run --tty --interactive kalilinux/kali-rolling
# start kali linux
docker run -it --name my-fedora-container fedora
# start fedora
sudo docker run -it fedora python3
## will start fedora and run "python3" command in fedora
sudo docker run -it fedora python3
sudo docker run -it fedora ls -la # will run fedora and execute ls -la 
# command inside fedora

   ![](s8.png)


**default tag is latest->latest version**

<img src="s9.png" widhth=100 height=100>

<img src="s10.png" widhth=100 height=100>

![image.png](s11.png)

***in previous command the created databases will be deleted when we rm the container. to keep them persistent we can use the second option***

In [ ]:
sudo docker inspect fedora # show details infos about the container
sudo docker logs python-hello-world # show the logs

<div>
    <img src="s12.png" width=400 height=400>
    <img src="s13.png" width=400 height=400>
    <img src="s14.png" width=400 height=400>
</div>

![](15.png)

![](s15.png)